## Pyridine Adsorption Energy Example

In [40]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/2_6difluoropyridine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [41]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_oct_2_2_6difluoropyridine_5_hydride_1_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

$$\Delta E_{ads} = (E_{complex+H}+E_{separate ligand}) - E_{complex} - E_H$$
### We need to add the energy of the separate ligand because in the hydride complex, we remove a ligand to add the hydrogen.

In [42]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "Fc1cccc(F)n1"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='FC5FNH3', pbc=False)


## Compute the Energy of the Complex

In [43]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:37:35  -113965.758026       13.388060
FIRE:    1 19:37:35  -113971.107330        9.054475
FIRE:    2 19:37:35  -113974.489258        5.663540
FIRE:    3 19:37:35  -113976.622777        3.830898
FIRE:    4 19:37:35  -113978.194831        3.670658
FIRE:    5 19:37:35  -113979.564887        3.055082
FIRE:    6 19:37:35  -113980.764397        3.052824
FIRE:    7 19:37:35  -113981.798261        2.889482
FIRE:    8 19:37:36  -113982.749827        2.645646
FIRE:    9 19:37:36  -113983.636849        2.799871
FIRE:   10 19:37:36  -113984.461243        2.448212
FIRE:   11 19:37:36  -113985.284348        2.972845
FIRE:   12 19:37:36  -113986.081967        2.708278
FIRE:   13 19:37:36  -113986.734390        1.837113
FIRE:   14 19:37:36  -113987.283517        1.814200
FIRE:   15 19:37:36  -113987.856282        2.062816
FIRE:   16 19:37:36  -113988.401614        1.470576
FIRE:   17 19:37:36  -113988.917477        1.875840
FIRE:   18 19:

In [44]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:37:46  -101830.823185       13.485259
FIRE:    1 19:37:46  -101835.448136        8.610290
FIRE:    2 19:37:46  -101838.122465        4.981582
FIRE:    3 19:37:47  -101839.716842        3.382394
FIRE:    4 19:37:47  -101840.931464        3.319123
FIRE:    5 19:37:47  -101841.975349        3.267246
FIRE:    6 19:37:47  -101842.812068        3.110770
FIRE:    7 19:37:47  -101843.529100        2.583975
FIRE:    8 19:37:47  -101844.240364        2.083090
FIRE:    9 19:37:47  -101844.922978        2.500426
FIRE:   10 19:37:47  -101845.593075        3.465517
FIRE:   11 19:37:47  -101846.279106        2.824057
FIRE:   12 19:37:47  -101846.825277        1.630456
FIRE:   13 19:37:47  -101847.251448        2.332886
FIRE:   14 19:37:48  -101847.704508        2.268563
FIRE:   15 19:37:48  -101848.117586        1.781751
FIRE:   16 19:37:48  -101848.545317        2.068580
FIRE:   17 19:37:48  -101848.947926        1.063707
FIRE:   18 19:

In [45]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:37:56   -12158.484558        2.733933
FIRE:    1 19:37:56   -12158.649560        1.616153
FIRE:    2 19:37:56   -12158.762721        1.439182
FIRE:    3 19:37:56   -12158.819426        1.481200
FIRE:    4 19:37:56   -12158.835579        1.115152
FIRE:    5 19:37:56   -12158.854775        0.479626
FIRE:    6 19:37:56   -12158.862552        0.259002
FIRE:    7 19:37:57   -12158.862851        0.250925
FIRE:    8 19:37:57   -12158.863414        0.240539
FIRE:    9 19:37:57   -12158.864177        0.226994
FIRE:   10 19:37:57   -12158.865067        0.212584
FIRE:   11 19:37:57   -12158.866013        0.199901
FIRE:   12 19:37:57   -12158.866967        0.191011
FIRE:   13 19:37:57   -12158.867912        0.186427
FIRE:   14 19:37:57   -12158.868955        0.184537
FIRE:   15 19:37:57   -12158.870128        0.182283
FIRE:   16 19:37:58   -12158.871457        0.174582
FIRE:   17 19:37:58   -12158.872915        0.157572
FIRE:   18 19:

## Compute the energy of the Hydrogen atom

In [46]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:38:00      -31.589716        0.047594
FIRE:    1 19:38:00      -31.589744        0.012353
FIRE:    2 19:38:00      -31.589733        0.031727
FIRE:    3 19:38:00      -31.589737        0.025917
FIRE:    4 19:38:00      -31.589743        0.015339
FIRE:    5 19:38:01      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [47]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: 0.158 eV
